In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
# sklearn tools for model training and assesment
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import GridSearchCV

In [4]:
test_df = test
train_df = train

test_ID = test_df['ID_code'].values
train_df = train_df.drop("ID_code",axis=1)
test_df = test_df.drop("ID_code",axis=1)

In [5]:
target_col = "target"

In [6]:
params = {
        "objective" : "binary",
        "metric" : "auc",
        "max_depth" : -1,
        "num_leaves" : 8,
        "min_data_in_leaf" : 25,
        "learning_rate" : 0.006,
        "bagging_fraction" : 0.2,
        "feature_fraction" : 0.4,
        "bagging_freq" : 1,
        "lambda_l1" : 5,
        "lambda_l2" : 5,
        "verbosity" : 1,
        "max_bin": 512,
        "num_threads" : 6
    }

In [7]:
# Create parameters to search
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [2000],
    'num_leaves': [4,2],
    'colsample_bytree' : [0.7,0.07],
    'reg_alpha' : [1,1.2,7],
    'reg_lambda' : [5,5.5,0.6],
    'subsample' : [0.7,0.07],
    'objective' : ['binary'],
    'boosting_type' : ['gbdt'],
}

In [8]:
train_y = train_df[target_col].values
features = [c for c in train_df.columns if c not in ['target']]
train_df = train_df.drop("target",axis=1)
train_X = train_df
test_X = test_df

In [9]:
lgtrain = lgb.Dataset(train_X, label=train_y)
#lgval = lgb.Dataset(val_X, label=val_y)
evals_result = {}

In [10]:
# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = 6, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'])

In [11]:
# Create the grid
grid = GridSearchCV(mdl, gridParams,scoring='roc_auc',
                    verbose=0,
                    cv=4,
                    n_jobs=8)

In [12]:
# Run the grid
grid.fit(train_X, train_y)

C:\Users\Ayushi.Goel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\Ayushi.Goel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\Ayushi.Goel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\Ayushi.

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_bin=512,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=6, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'learning_rate': [0.005], 'n_estimators': [2000], 'num_leaves': [4, 2], 'colsample_bytree': [0.7, 0.07], 'reg_alpha': [1, 1.2, 7], 'reg_lambda': [5, 5.5, 0.6], 'subsample': [0.7, 0.07], 'objective': ['binary'], 'boosting_type': ['gbdt']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'boosting_type': 'gbdt', 'colsample_bytree': 0.07, 'learning_rate': 0.005, 'n_estimators': 2000, 'num_leaves': 4, 'objective': 'binary', 'reg_alpha': 1.2, 'reg_lambda': 0.6, 'subsample': 0.7}
0.8403487192000508


In [14]:
params['feature_fraction'] = grid.best_params_['colsample_bytree']
params['learning_rate'] = grid.best_params_['learning_rate']
# params['max_bin'] = grid.best_params_['max_bin']
params['num_leaves'] = grid.best_params_['num_leaves']
params['lambda_l1'] = grid.best_params_['reg_alpha']
params['lambda_l2'] = grid.best_params_['reg_lambda']
params['bagging_fraction'] = grid.best_params_['subsample']
numberOfEstimators = grid.best_params_['n_estimators']

In [15]:
def run_lgb(train_X, train_y, val_X, val_y, test_X,params):    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 300000, valid_sets=[lgtrain,lgval], early_stopping_rounds=500, verbose_eval=100, evals_result=evals_result)
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [16]:
pred_test = 0
kf = model_selection.KFold(n_splits=6, random_state=2018, shuffle=True)
feature_importance_df = pd.DataFrame()
for dev_index, val_index in kf.split(train_df):
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_test_tmp, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, test_X,params)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = model.feature_importance()
    #fold_importance_df["fold"] = dev_index + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    pred_test += pred_test_tmp / 6

Training until validation scores don't improve for 500 rounds.
[100]	training's auc: 0.812493	valid_1's auc: 0.804858
[200]	training's auc: 0.814449	valid_1's auc: 0.80524
[300]	training's auc: 0.820243	valid_1's auc: 0.810761
[400]	training's auc: 0.822522	valid_1's auc: 0.813774
[500]	training's auc: 0.824764	valid_1's auc: 0.815334
[600]	training's auc: 0.827894	valid_1's auc: 0.81825
[700]	training's auc: 0.831338	valid_1's auc: 0.821569
[800]	training's auc: 0.832916	valid_1's auc: 0.823159
[900]	training's auc: 0.835107	valid_1's auc: 0.824953
[1000]	training's auc: 0.837011	valid_1's auc: 0.826837
[1100]	training's auc: 0.839236	valid_1's auc: 0.828744
[1200]	training's auc: 0.841113	valid_1's auc: 0.83074
[1300]	training's auc: 0.843289	valid_1's auc: 0.832713
[1400]	training's auc: 0.844692	valid_1's auc: 0.834061
[1500]	training's auc: 0.846323	valid_1's auc: 0.835368
[1600]	training's auc: 0.847814	valid_1's auc: 0.836751
[1700]	training's auc: 0.849386	valid_1's auc: 0.8383

[14700]	training's auc: 0.909953	valid_1's auc: 0.891715
[14800]	training's auc: 0.910069	valid_1's auc: 0.891804
[14900]	training's auc: 0.910213	valid_1's auc: 0.891918
[15000]	training's auc: 0.910336	valid_1's auc: 0.892003
[15100]	training's auc: 0.910463	valid_1's auc: 0.892105
[15200]	training's auc: 0.910568	valid_1's auc: 0.892185
[15300]	training's auc: 0.910686	valid_1's auc: 0.892281
[15400]	training's auc: 0.910809	valid_1's auc: 0.892368
[15500]	training's auc: 0.910922	valid_1's auc: 0.892471
[15600]	training's auc: 0.911034	valid_1's auc: 0.892559
[15700]	training's auc: 0.911159	valid_1's auc: 0.892628
[15800]	training's auc: 0.911281	valid_1's auc: 0.892711
[15900]	training's auc: 0.91139	valid_1's auc: 0.892804
[16000]	training's auc: 0.911507	valid_1's auc: 0.892881
[16100]	training's auc: 0.911628	valid_1's auc: 0.892962
[16200]	training's auc: 0.911747	valid_1's auc: 0.893052
[16300]	training's auc: 0.911858	valid_1's auc: 0.893125
[16400]	training's auc: 0.911966

[29200]	training's auc: 0.921174	valid_1's auc: 0.898014
[29300]	training's auc: 0.921228	valid_1's auc: 0.898031
[29400]	training's auc: 0.921285	valid_1's auc: 0.898046
[29500]	training's auc: 0.92134	valid_1's auc: 0.898052
[29600]	training's auc: 0.921395	valid_1's auc: 0.898074
[29700]	training's auc: 0.92145	valid_1's auc: 0.898076
[29800]	training's auc: 0.921505	valid_1's auc: 0.898077
[29900]	training's auc: 0.921559	valid_1's auc: 0.898093
[30000]	training's auc: 0.921615	valid_1's auc: 0.898106
[30100]	training's auc: 0.921677	valid_1's auc: 0.898112
[30200]	training's auc: 0.921732	valid_1's auc: 0.898124
[30300]	training's auc: 0.92179	valid_1's auc: 0.89814
[30400]	training's auc: 0.921843	valid_1's auc: 0.898145
[30500]	training's auc: 0.9219	valid_1's auc: 0.898152
[30600]	training's auc: 0.921955	valid_1's auc: 0.89815
[30700]	training's auc: 0.922006	valid_1's auc: 0.898172
[30800]	training's auc: 0.922056	valid_1's auc: 0.898178
[30900]	training's auc: 0.92211	valid_

[4800]	training's auc: 0.879852	valid_1's auc: 0.871035
[4900]	training's auc: 0.880543	valid_1's auc: 0.871564
[5000]	training's auc: 0.88127	valid_1's auc: 0.872145
[5100]	training's auc: 0.881876	valid_1's auc: 0.8727
[5200]	training's auc: 0.882624	valid_1's auc: 0.873253
[5300]	training's auc: 0.883357	valid_1's auc: 0.873818
[5400]	training's auc: 0.884045	valid_1's auc: 0.874388
[5500]	training's auc: 0.884623	valid_1's auc: 0.874837
[5600]	training's auc: 0.885184	valid_1's auc: 0.875264
[5700]	training's auc: 0.885726	valid_1's auc: 0.875657
[5800]	training's auc: 0.886311	valid_1's auc: 0.876093
[5900]	training's auc: 0.886822	valid_1's auc: 0.87648
[6000]	training's auc: 0.887408	valid_1's auc: 0.876898
[6100]	training's auc: 0.887901	valid_1's auc: 0.87727
[6200]	training's auc: 0.888393	valid_1's auc: 0.877637
[6300]	training's auc: 0.888902	valid_1's auc: 0.877955
[6400]	training's auc: 0.889445	valid_1's auc: 0.87837
[6500]	training's auc: 0.889918	valid_1's auc: 0.87874

[19400]	training's auc: 0.915074	valid_1's auc: 0.895158
[19500]	training's auc: 0.915155	valid_1's auc: 0.895204
[19600]	training's auc: 0.915234	valid_1's auc: 0.895249
[19700]	training's auc: 0.915316	valid_1's auc: 0.895302
[19800]	training's auc: 0.915398	valid_1's auc: 0.895345
[19900]	training's auc: 0.915482	valid_1's auc: 0.895383
[20000]	training's auc: 0.91556	valid_1's auc: 0.895417
[20100]	training's auc: 0.915651	valid_1's auc: 0.895452
[20200]	training's auc: 0.915725	valid_1's auc: 0.895483
[20300]	training's auc: 0.915802	valid_1's auc: 0.89552
[20400]	training's auc: 0.915878	valid_1's auc: 0.895564
[20500]	training's auc: 0.915956	valid_1's auc: 0.895615
[20600]	training's auc: 0.91603	valid_1's auc: 0.895655
[20700]	training's auc: 0.916108	valid_1's auc: 0.895702
[20800]	training's auc: 0.91618	valid_1's auc: 0.895727
[20900]	training's auc: 0.91625	valid_1's auc: 0.895758
[21000]	training's auc: 0.916319	valid_1's auc: 0.895793
[21100]	training's auc: 0.916403	val

...

[5900]	training's auc: 0.886024	valid_1's auc: 0.877096
[6000]	training's auc: 0.88654	valid_1's auc: 0.877527
[6100]	training's auc: 0.886994	valid_1's auc: 0.877927
[6200]	training's auc: 0.887497	valid_1's auc: 0.87838
[6300]	training's auc: 0.887977	valid_1's auc: 0.878772
[6400]	training's auc: 0.888461	valid_1's auc: 0.879215
[6500]	training's auc: 0.888939	valid_1's auc: 0.879618
[6600]	training's auc: 0.889377	valid_1's auc: 0.879979
[6700]	training's auc: 0.889822	valid_1's auc: 0.880375
[6800]	training's auc: 0.890277	valid_1's auc: 0.880756
[6900]	training's auc: 0.890701	valid_1's auc: 0.881111
[7000]	training's auc: 0.891094	valid_1's auc: 0.881486
[7100]	training's auc: 0.891484	valid_1's auc: 0.881821
[7200]	training's auc: 0.891874	valid_1's auc: 0.882149
[7300]	training's auc: 0.892279	valid_1's auc: 0.882513
[7400]	training's auc: 0.892657	valid_1's auc: 0.882839
[7500]	training's auc: 0.893059	valid_1's auc: 0.883178
[7600]	training's auc: 0.893443	valid_1's auc: 0.8

[20400]	training's auc: 0.915108	valid_1's auc: 0.899235
[20500]	training's auc: 0.915184	valid_1's auc: 0.899279
[20600]	training's auc: 0.91526	valid_1's auc: 0.899324
[20700]	training's auc: 0.915346	valid_1's auc: 0.899362
[20800]	training's auc: 0.915421	valid_1's auc: 0.89942
[20900]	training's auc: 0.915494	valid_1's auc: 0.899452
[21000]	training's auc: 0.915573	valid_1's auc: 0.899497
[21100]	training's auc: 0.915648	valid_1's auc: 0.899526
[21200]	training's auc: 0.915718	valid_1's auc: 0.899565
[21300]	training's auc: 0.91579	valid_1's auc: 0.899599
[21400]	training's auc: 0.915866	valid_1's auc: 0.899628
[21500]	training's auc: 0.915935	valid_1's auc: 0.899656
[21600]	training's auc: 0.916008	valid_1's auc: 0.899681
[21700]	training's auc: 0.916085	valid_1's auc: 0.89971
[21800]	training's auc: 0.916162	valid_1's auc: 0.89974
[21900]	training's auc: 0.916232	valid_1's auc: 0.89976
[22000]	training's auc: 0.916303	valid_1's auc: 0.899809
[22100]	training's auc: 0.916377	vali

[600]	training's auc: 0.827916	valid_1's auc: 0.819681
[700]	training's auc: 0.831168	valid_1's auc: 0.823131
[800]	training's auc: 0.832961	valid_1's auc: 0.824275
[900]	training's auc: 0.835214	valid_1's auc: 0.826173
[1000]	training's auc: 0.836802	valid_1's auc: 0.82747
[1100]	training's auc: 0.839067	valid_1's auc: 0.829385
[1200]	training's auc: 0.841143	valid_1's auc: 0.830966
[1300]	training's auc: 0.843339	valid_1's auc: 0.833023
[1400]	training's auc: 0.84478	valid_1's auc: 0.834341
[1500]	training's auc: 0.846307	valid_1's auc: 0.83561
[1600]	training's auc: 0.847827	valid_1's auc: 0.837016
[1700]	training's auc: 0.849311	valid_1's auc: 0.838462
[1800]	training's auc: 0.85068	valid_1's auc: 0.839651
[1900]	training's auc: 0.851851	valid_1's auc: 0.840752
[2000]	training's auc: 0.853249	valid_1's auc: 0.84196
[2100]	training's auc: 0.854188	valid_1's auc: 0.842807
[2200]	training's auc: 0.855524	valid_1's auc: 0.844173
[2300]	training's auc: 0.856561	valid_1's auc: 0.845168
[

[15300]	training's auc: 0.909942	valid_1's auc: 0.896548
[15400]	training's auc: 0.910071	valid_1's auc: 0.896669
[15500]	training's auc: 0.910179	valid_1's auc: 0.896735
[15600]	training's auc: 0.910295	valid_1's auc: 0.89682
[15700]	training's auc: 0.910415	valid_1's auc: 0.896884
[15800]	training's auc: 0.910521	valid_1's auc: 0.896987
[15900]	training's auc: 0.910643	valid_1's auc: 0.897109
[16000]	training's auc: 0.91075	valid_1's auc: 0.897214
[16100]	training's auc: 0.910856	valid_1's auc: 0.897315
[16200]	training's auc: 0.910973	valid_1's auc: 0.897417
[16300]	training's auc: 0.911086	valid_1's auc: 0.897514
[16400]	training's auc: 0.911196	valid_1's auc: 0.897594
[16500]	training's auc: 0.911297	valid_1's auc: 0.897684
[16600]	training's auc: 0.911405	valid_1's auc: 0.89778
[16700]	training's auc: 0.911501	valid_1's auc: 0.897864
[16800]	training's auc: 0.91161	valid_1's auc: 0.897947
[16900]	training's auc: 0.911701	valid_1's auc: 0.898022
[17000]	training's auc: 0.911803	va

[29800]	training's auc: 0.920646	valid_1's auc: 0.903265
[29900]	training's auc: 0.9207	valid_1's auc: 0.903285
[30000]	training's auc: 0.920754	valid_1's auc: 0.903315
[30100]	training's auc: 0.920809	valid_1's auc: 0.903321
[30200]	training's auc: 0.920861	valid_1's auc: 0.903336
[30300]	training's auc: 0.920921	valid_1's auc: 0.903353
[30400]	training's auc: 0.920975	valid_1's auc: 0.903364
[30500]	training's auc: 0.92103	valid_1's auc: 0.903375
[30600]	training's auc: 0.921082	valid_1's auc: 0.903391
[30700]	training's auc: 0.921136	valid_1's auc: 0.903395
[30800]	training's auc: 0.92119	valid_1's auc: 0.903411
[30900]	training's auc: 0.921241	valid_1's auc: 0.903429
[31000]	training's auc: 0.921296	valid_1's auc: 0.903444
[31100]	training's auc: 0.921354	valid_1's auc: 0.903463
[31200]	training's auc: 0.921413	valid_1's auc: 0.903478
[31300]	training's auc: 0.921463	valid_1's auc: 0.903493
[31400]	training's auc: 0.921517	valid_1's auc: 0.903523
[31500]	training's auc: 0.921572	va

In [17]:
print('save result.')
pd.DataFrame({'ID_code':test_ID,'target':pred_test}).to_csv('subgridcv.csv',index=False)
print('done.')

save result.
done.
